In [1]:
import pandas as pd
import sqlite3

In [2]:
con = sqlite3.connect("../instance/database.db")

In [3]:
# Load the data into a DataFrame
players = pd.read_sql_query("SELECT * from Player", con)
players

,id,first_name,last_name,gender
0,1,Rogozin,Anton,male
1,2,Razzakov,Alisher,male
2,3,Larionov,Svyatoslav,male
3,4,Shirinov,Jahon,male
4,5,Shamuratov,Farrukh,male
5,6,Musadjanov,Hatam,male
6,7,Malikov,Humoyun,male
7,8,Kamilov,Baxriddin,male
8,9,Vohidov,Nodir,male


In [4]:
results = pd.read_sql_query("SELECT * from Result", con)
results

,id,player_id,position,match_count,win_count,tie_win_count,set_diff,game_diff,division_id,relegation
0,1,1,1,8,7,0,11,46,1,unchanged
1,2,2,2,6,6,0,12,37,1,unchanged
2,3,3,3,6,4,0,5,18,1,unchanged
3,4,4,4,5,4,0,3,-1,1,unchanged
4,5,5,5,7,3,1,-2,-17,1,unchanged
5,6,6,6,8,3,0,-4,3,1,relegated
6,7,7,7,6,2,0,-3,-11,1,relegated
7,8,8,8,6,0,0,-11,-33,1,relegated
8,9,9,9,6,0,0,-11,-42,1,relegated


In [5]:
league = pd.read_sql_query("SELECT * from League", con)
league

,id,name
0,1,Tashkent Masters


In [6]:
season = pd.read_sql_query("SELECT * from Season", con)
season

,id,name,year,date_start,date_end,league_id
0,1,1,2025,None,None,1


In [7]:
divisions = pd.read_sql_query("SELECT * from Division", con)
divisions

,id,name,priority,season_id
0,1,M1,10,1


In [8]:
df = pd.read_csv('./Tashkent Masters League - results.csv')
df

,year,season,division,last_name,first_name,position,match_count,win_count,tie_win_count,set_diff,game_diff,relegation
0,2025,1,M1,Rogozin,Anton,1,8,7,0,11,46,unchanged
1,2025,1,M1,Razzakov,Alisher,2,6,6,0,12,37,unchanged
2,2025,1,M1,Larionov,Svyatoslav,3,6,4,0,5,18,unchanged
3,2025,1,M1,Shirinov,Jahon,4,5,4,0,3,-1,unchanged
4,2025,1,M1,Shamuratov,Farrukh,5,7,3,1,-2,-17,unchanged
...,...,...,...,...,...,...,...,...,...,...,...,...
171,2024,1,M1b,Magomedov,Arsen,4,2,0,0,-3,-11,relegated
172,2024,1,M1b,Xaleulin,Ruslan,5,1,0,0,-1,-2,relegated
173,2024,1,M1b,Logvis,Kirill,6,1,0,0,-1,-4,relegated
174,2024,1,M1b,Zakharchenko,Petro,7,1,0,0,-2,-7,relegated


In [9]:
df['full_name'] = df['last_name'] + ' ' + df['first_name']

In [10]:
df['full_name'].value_counts()

Rogozin Anton        6
Kamilov Baxriddin    6
Razzakov Alisher     6
Tadjiev Khamdam      6
Malikov Humoyun      5
                    ..
Neklesov Igor        1
Hayitov Olimjon      1
N Dzhalil            1
Ismailov Ziyodbek    1
Xazratov Akmal       1
Name: full_name, Length: 76, dtype: int64

In [11]:
df['full_name'].nunique()

76

In [12]:
new_players = df[['full_name', 'first_name', 'last_name']].drop_duplicates().sort_values('full_name')
new_players = new_players.reset_index().reset_index()
new_players['id'] = new_players['level_0'] + 1
new_players['gender'] = 'male'
new_players = new_players[['id', 'first_name', 'last_name', 'gender']]
new_players

,id,first_name,last_name,gender
0,1,Sharof,Abdullaev,male
1,2,Vladimir,Abduraimov,male
2,3,Alisher,Abduvakhidov,male
3,4,Izzat,Ahrarov,male
4,5,Farrukh,Akhtamov,male
...,...,...,...,...
71,72,Sasha,Wao,male
72,73,Ruslan,Xaleulin,male
73,74,Akmal,Xazratov,male
74,75,Petro,Zakharchenko,male


In [13]:
# Create DataFrame with string dates, then convert to datetime
data = {
    'start_date': ['2024-06-01', '2024-07-01', '2024-08-01', '2024-09-23', '2025-04-05', '2025-02-22'],
    'end_date': ['2024-07-01', '2024-08-01', '2024-09-15', '2024-11-23', '2025-05-18', '2025-03-30']
}

dates = pd.DataFrame(data)

# Convert string dates to datetime objects
dates['start_date'] = pd.to_datetime(dates['start_date'])
dates['end_date'] = pd.to_datetime(dates['end_date'])
dates

,start_date,end_date
0,2024-06-01,2024-07-01
1,2024-07-01,2024-08-01
2,2024-08-01,2024-09-15
3,2024-09-23,2024-11-23
4,2025-04-05,2025-05-18
5,2025-02-22,2025-03-30


In [14]:
new_seasons = df[['year', 'season']].drop_duplicates().sort_values(['year', 'season'])
new_seasons = new_seasons.reset_index(drop=True).join(dates).sort_values('start_date').reset_index(drop=True)
new_seasons['id'] = new_seasons.index + 1
new_seasons['league_id'] = 1
new_seasons = new_seasons.rename(columns={'season': 'name'})
new_seasons

,year,name,start_date,end_date,id,league_id
0,2024,1,2024-06-01,2024-07-01,1,1
1,2024,2,2024-07-01,2024-08-01,2,1
2,2024,3,2024-08-01,2024-09-15,3,1
3,2024,4,2024-09-23,2024-11-23,4,1
4,2025,Preseason,2025-02-22,2025-03-30,5,1
5,2025,1,2025-04-05,2025-05-18,6,1


In [15]:
def calc_priority(name):
    if name.startswith('M1'):
        return 110
    if name.startswith('M2'):
        return 120
    if name.startswith('M3'):
        return 130
    if name.startswith('M4'):
        return 140
    if name.startswith('O1'):
        return 210
    if name.startswith('O2'):
        return 220
    if name == 'Masters':
         return 150
    if name == 'Open':
        return 250

In [16]:
new_divisions = df[['year', 'season', 'division']]
new_divisions = new_divisions.join(new_seasons.set_index(['year', 'name']), on=['year', 'season']).rename(columns={'id': 'season_id'})
new_divisions = new_divisions.sort_values(['season_id', 'division']).drop_duplicates()[['division', 'season_id']]
new_divisions = new_divisions.reset_index(drop=True)
new_divisions['id'] = new_divisions.index + 1
new_divisions['priority'] = new_divisions['division'].apply(calc_priority)
new_divisions = new_divisions.rename(columns={'division': 'name'})
new_divisions

,name,season_id,id,priority
0,M1a,1,1,110
1,M1b,1,2,110
2,M1,2,3,110
3,M2,2,4,120
4,M1,3,5,110
5,M2,3,6,120
6,O1,3,7,210
7,M1,4,8,110
8,M2,4,9,120
9,M3,4,10,130


In [17]:
new_results = df.join(new_seasons.set_index(['year', 'name']), on=['year', 'season']).rename(columns={'id': 'season_id'})
new_results = new_results.join(new_divisions.set_index(['season_id', 'name']), on=['season_id', 'division']).rename(columns={'id': 'division_id'})
new_results = new_results.join(new_players.set_index(['first_name', 'last_name']), on=['first_name', 'last_name']).rename(columns={'id': 'player_id'})
new_results = new_results.sort_values(['season_id', 'division_id', 'position'])
new_results = new_results.reset_index(drop=True)
new_results['id'] = new_results.index + 1
new_results = new_results[['id', 'player_id', 'position', 'match_count', 'win_count', 'tie_win_count', 'set_diff', 'game_diff',
             'division_id', 'relegation']]
new_results

,id,player_id,position,match_count,win_count,tie_win_count,set_diff,game_diff,division_id,relegation
0,1,60,1,7,6,1,8,27,1,unchanged
1,2,46,2,7,6,0,8,17,1,unchanged
2,3,58,3,6,5,0,9,33,1,unchanged
3,4,71,4,7,3,0,-1,-4,1,unchanged
4,5,37,5,5,2,0,-2,-4,1,unchanged
...,...,...,...,...,...,...,...,...,...,...
171,172,53,2,2,2,1,2,11,19,promoted
172,173,49,3,4,2,0,0,-5,19,unchanged
173,174,15,4,3,1,1,-1,-9,19,unchanged
174,175,28,5,5,1,0,-3,-9,19,unchanged


In [20]:
new_results.relegation.drop_duplicates()

0            unchanged
7            relegated
25            promoted
90       fast promoted
149    double promoted
Name: relegation, dtype: object

In [18]:
# Write the new DataFrame to a new SQLite table
new_players.to_sql("Player", con, if_exists="replace")
new_seasons.to_sql("Season", con, if_exists="replace")
new_divisions.to_sql("Division", con, if_exists="replace")
new_results.to_sql("Result", con, if_exists="replace")

176

In [19]:


con.close()